<a href="https://colab.research.google.com/github/srikanthpurimitla/Python-/blob/main/updatedProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install python-docx
import random
import docx
import os

def create_timetable_doc(title, timetable, filename):
    """Creates and saves a timetable document."""
    doc = docx.Document()
    doc.add_heading(title, 0)

    table = doc.add_table(rows=1, cols=7)
    table.style = "Table Grid"
    hdr_cells = table.rows[0].cells

    time_slots = ["7:30-8:20", "8:20-9:10", "9:10-10:00","10:30-11:20", "11:20-12:10", "12:10-1:00"]
    for i, slot in enumerate(time_slots):
        hdr_cells[i + 1].text = slot

    days = ["MON", "TUE", "WED", "THU", "FRI", "SAT"]
    for day_index, day in enumerate(days):
        row_cells = table.add_row().cells
        row_cells[0].text = day
        for period_index, period in enumerate(timetable[day_index]):
            if period:
                row_cells[period_index + 1].text = f"{period[0]} ({period[1]})"

    doc.save(filename)
    os.system(f"start {filename}")

def assign_first_periods(tt, teachers, teacher_subject_index, teacher_limit_index):
    """Ensures each teacher gets at least one first-period slot, excluding HODs and Principals."""
    assigned_teachers = set()
    days = len(tt)

    for day in range(days):
        for teacher in teachers:
            if ("HOD" in teacher[0] or "Principal" in teacher[0]):  # Skip HODs and Principals
                continue
            if teacher[teacher_limit_index] > 0 and teacher[0] not in assigned_teachers:
                tt[day][0].extend([teacher[teacher_subject_index], teacher[0]])
                teacher[teacher_limit_index] -= 1
                assigned_teachers.add(teacher[0])
                break

def can_assign_teacher(tt, teacher_name, subject, day, period):
    """Check if the teacher is already assigned at the same time in another class or consecutively in the same class."""
    # Check if the teacher is assigned in another class at the same time
    for ttable in [first_tt, second_tt, three_tt, four_tt, five_tt, six_tt, seven_tt, eight_tt, nine_tt, ten_tt]:
        if ttable[day][period] and ttable[day][period][1] == teacher_name:
            return False  # The teacher is already teaching in another class at this time

    # Check if the teacher is teaching the same subject consecutively in the same class
    if period > 0 and tt[day][period - 1] and tt[day][period - 1][1] == teacher_name:
        return False  # Teacher is teaching back-to-back periods

    # Check if the teacher already taught the same subject in this class today
    for p in range(6):  # Assuming 6 periods per day
        if tt[day][p] and tt[day][p][1] == teacher_name and tt[day][p][0] == subject:
            return False  # The teacher already taught this subject in this class today

    return True

def arrange_timetable(tt, teacher_subject_index, teacher_limit_index, noofteachers, teachers):
    """Arranges a timetable ensuring teachers are assigned correctly based on constraints."""
    assign_first_periods(tt, teachers, teacher_subject_index, teacher_limit_index)

    for day in range(6):  # Iterate over days
        for period in range(6):  # Iterate over periods
            if not tt[day][period]:  # If the period is empty
                random.shuffle(teachers)  # Shuffle to ensure randomness
                for teacher in teachers:
                    if teacher[teacher_limit_index] > 0:
                        teacher_name = teacher[0]  # Get teacher's name
                        subject = teacher[teacher_subject_index]  # Get subject

                        if ("Principal" in teacher_name or "HOD" in teacher_name) and (period == 0 or period == 5):
                            continue  # Skip assigning to 1st and 6th period

                        if can_assign_teacher(tt, teacher_name, subject, day, period):
                            teacher[teacher_limit_index] -= 1
                            tt[day][period].extend([subject, teacher_name])
                            break  # Move to the next period



# Input Data
noofteachers = 26
teachers = [["Mr.B.MANIKANTA","C","PYTHON",2,3,"102","",0,0,"","",3,0,"","PYTHON",2,3,"103","",0,0,"","",0,0,"","EITK",2,0,"","EH&SE",4,3,"602","EITK",2,0,"","",0,0,""],
["Mr.V.CHAKRADHAR HOD","C","",0,0,"","",0,0,"","OS",4,0,"","OS",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mrs. R. HIMABINDU","A","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","ML",4,3,"401","ML",4,3,"501","",0,0,"","",0,0,"","NLP",4,3,"701"],
["Mr. P.V. NAGA SRINIVAS","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","ATFL",4,0,"","ATFL",4,0,"","",0,0,""],
["Mr. D SEKHAR BABU" ,"B","",0,0,"","DS",4,3,"201","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.T N SIVA KUMAR","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","EP",3,3,"404","EP",3,3,"503","",0,0,"","",0,0,"","",0,0,""],
["Mr.V. ROSHAN KUMAR","D","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.MD.ISMAIL KHAN","A","CO",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","DHV",4,3,"702","",0,0,""],
["Mr.R. KARTHEEK","C","OOPS",4,3,"101","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","CCIDF",3,0,""],
["Mr.P. BHANU CHAND","C","",0,0,"","",0,0,"","DS",4,3,"301","",0,0,"","DS",4,3,201,"",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.N. VENKATESWARA RAO","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","DMDW",4,3,"403","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mrs.D. DWIJA","A","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","SE",4,3,"402","",0,0,"","SE",4,3,"601","",0,0,"","",0,0,""],
["Mrs.K. NEELIMA" ,"C","",0,0,"","OOPS",4,3,"202","",0,0,"","",0,0,"","OOPS",4,3,"102","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mrs.  I BHAVANA","A","",0,0,"","CO",4,0,"","",4,3,"","CO",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.K JOHN BUNYAN","A","",0,0,"","",0,0,"","",4,3,"","",0,0,"","",0,0,"","EITK",2,0,"","CN",3,0,"","EITK",2,0,"","CN",3,0,"","",0,0,""],
["Mrs. M. RISHITHA","B","",0,0,"","",0,0,"","",4,3,"","",0,0,"","",0,0,"","CN",3,0,"","",0,0,"","CN",3,0,"","",0,0,"","",0,0,""],
["Mrs. G.V. LEELAKUMARI","C","",0,0,"","",0,0,"","OOPS",4,3,"302","OOPS",4,3,"102","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr. M BABU RAO","A","DS",4,3,"101","",0,0,"","",0,0,"","DS",4,3,"101","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","BDA",4,3,"702"],
["Mr.M.VEERABRAHMAM","C","",0,0,"","",0,0,"","PE&HV",2,0,"","",0,0,"","PE&HV",2,0,"","",0,0,"","SE",4,3,"501","",0,0,"","SE",4,3,"701","",0,0,""],
["Dr.P.RAMESH BABU","A","OS",4,0,"","",0,0,"","",0,0,"","",0,0,"","OS",4,0,"","",0,0,"","DMDW",4,3,"503","",0,0,"","",0,0,"","",0,0,""],
["Mr.B.KISHORE BABU","C","",0,0,"","",0,0,"","CO",4,0,"","",0,0,"","CO",4,0,"","",0,0,"","",0,0,"","IISCL",3,0,"","IISCL",3,0,"","",0,0,""],
["Ms.ALLU ANEESHA","A","",0,0,"","PYTHON",2,3,"201","PYTHON",2,3,"302","",0,0,"","PYTHON",2,3,"103","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["P. BHAVANI","B","PE&HV",2,0,"","PE&HV",2,0,"","",0,0,"","PE&HV",2,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["MATH","A","P&S",4,0,"","P&S",4,0,"","P&S",4,0,"","P&S",4,0,"","P&S",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["IMED","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","IMED",4,0,""],
["OE","B","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","OE",4,0,""]]

# Timetables
first_tt = [[[], [], [], [], [], []] for _ in range(6)]
second_tt = [[[], [], [], [], [], []] for _ in range(6)]
three_tt = [[[], [], [], [], [], []] for _ in range(6)]
four_tt = [[[], [], [], [], [], []] for _ in range(6)]
five_tt = [[[], [], [], [], [], []] for _ in range(6)]
six_tt = [[[], [], [], [], [], []] for _ in range(6)]
seven_tt = [[[], [], [], [], [], []] for _ in range(6)]
eight_tt = [[[], [], [], [], [], []] for _ in range(6)]
nine_tt = [[[], [], [], [], [], []] for _ in range(6)]
ten_tt = [[[], [], [], [], [], []] for _ in range(6)]

# Arrange Timetables
arrange_timetable(first_tt, 2, 3, noofteachers, teachers)
arrange_timetable(second_tt, 6, 7, noofteachers, teachers)
arrange_timetable(three_tt, 10, 11, noofteachers, teachers)
arrange_timetable(four_tt, 14, 15, noofteachers, teachers)
arrange_timetable(five_tt, 18, 19, noofteachers, teachers)
arrange_timetable(six_tt, 22, 23, noofteachers, teachers)
arrange_timetable(seven_tt, 26, 27, noofteachers, teachers)
arrange_timetable(eight_tt, 30, 31, noofteachers, teachers)
arrange_timetable(nine_tt, 34, 35, noofteachers, teachers)
arrange_timetable(ten_tt, 38, 39, noofteachers, teachers)

# Create Word Documents
create_timetable_doc("1st Timetable", first_tt, "first_timetable.docx")
create_timetable_doc("2nd  Timetable", second_tt, "second_timetable.docx")
create_timetable_doc("3rd Timetable", three_tt, "three_timetable.docx")
create_timetable_doc("4th Timetable", four_tt, "four_timetable.docx")
create_timetable_doc("5th Timetable", five_tt, "five_timetable.docx")
create_timetable_doc("6th Timetable", six_tt, "six_timetable.docx")
create_timetable_doc("7th Timetable", seven_tt, "seven_timetable.docx")
create_timetable_doc("8th Timetable", eight_tt, "eight_timetable.docx")
create_timetable_doc("9th Timetable", nine_tt, "nine_timetable.docx")
create_timetable_doc("10th Timetable", ten_tt, "ten_timetable.docx")
